In [28]:
import tensorflow as tf
from tensorflow.keras import layers, models
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


In [29]:
def load_and_preprocess_data():
    (ds_train, ds_test), ds_info = tfds.load(
        'oxford_iiit_pet:4.0.0',
        split=['train[:80%]', 'train[80%:]'],
        with_info=True,
        as_supervised=True
    )

    def preprocess(image, mask):
        image = tf.cast(image, tf.float32) / 255.0
        mask = tf.cast(mask, tf.int32) - 1  # Маски: 1, 2, 3 -> 0, 1, 2
        image = tf.image.resize(image, [128, 128])
        mask = tf.expand_dims(mask, axis=-1)  # Добавляем канал: [H, W] -> [H, W, 1]
        mask = tf.image.resize(mask, [128, 128], method='nearest')  # Изменяем размер
        mask = tf.squeeze(mask, axis=-1)  # Убираем канал: [H, W, 1] -> [H, W]
        return image, mask

    ds_train = ds_train.map(preprocess).cache().shuffle(1000).batch(8).prefetch(tf.data.AUTOTUNE)
    ds_test = ds_test.map(preprocess).batch(8).prefetch(tf.data.AUTOTUNE)
    return ds_train, ds_test, ds_info

In [30]:
# Dice Loss
def dice_loss(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    intersection = tf.reduce_sum(y_true * y_pred, axis=[1, 2, 3])
    union = tf.reduce_sum(y_true, axis=[1, 2, 3]) + tf.reduce_sum(y_pred, axis=[1, 2, 3])
    return 1 - (2. * intersection + 1.) / (union + 1.)

In [31]:

# Комбинированная функция потерь
def combined_loss(y_true, y_pred):
    ce_loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)(y_true, y_pred)
    d_loss = dice_loss(tf.one_hot(y_true, depth=3), y_pred)
    return ce_loss + d_loss

In [32]:

# Блок энкодера
def encoder_block(inputs, filters):
    x = layers.Conv2D(filters, 3, padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU(alpha=0.1)(x)
    x = layers.Conv2D(filters, 3, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU(alpha=0.1)(x)
    p = layers.MaxPooling2D(2)(x)
    return x, p


In [33]:
# Блок декодера
def decoder_block(inputs, skip, filters):
    x = layers.UpSampling2D(2)(inputs)
    x = layers.Concatenate()([x, skip])
    x = layers.Conv2D(filters, 3, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU(alpha=0.1)(x)
    x = layers.Conv2D(filters, 3, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU(alpha=0.1)(x)
    return x


In [34]:
 # Создание U-Net
def build_unet():
    inputs = layers.Input(shape=(128, 128, 3))

 # Энкодер
    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)

    # Бутылочное горлышко
    b = layers.Conv2D(1024, 3, padding='same')(p4)
    b = layers.BatchNormalization()(b)
    b = layers.LeakyReLU(alpha=0.1)(b)
    b = layers.Conv2D(1024, 3, padding='same')(b)
    b = layers.BatchNormalization()(b)
    b = layers.LeakyReLU(alpha=0.1)(b)

    # Декодер
    d1 = decoder_block(b, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)


In [35]:
# Выходной слой
    outputs = layers.Conv2D(3, 1, padding='same', activation='softmax')(d4)

    model = models.Model(inputs, outputs)
    return model


IndentationError: unexpected indent (ipython-input-35-1596335256.py, line 2)

In [ ]:
# Основной код
if __name__ == '__main__':
    # Загрузка данных
    ds_train, ds_test, ds_info = load_and_preprocess_data()

    # Создание и компиляция модели
    model = build_unet()
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
        loss=combined_loss,
        metrics=[tf.keras.metrics.MeanIoU(num_classes=3)]
    )

In [ ]:
# Коллбэки
    callbacks = [
        EarlyStopping(patience=5, restore_best_weights=True),
        ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_mean_io_u', mode='max')
    ]


In [ ]:
 # Обучение
    history = model.fit(
        ds_train,
        validation_data=ds_test,
        epochs=20,
        callbacks=callbacks
    )

In [ ]:
 # Визуализация IoU
    plt.figure(figsize=(8, 6))
    plt.plot(history.history['mean_io_u'], label='Train IoU')
    plt.plot(history.history['val_mean_io_u'], label='Val IoU')
    plt.title('Mean IoU over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Mean IoU')
    plt.legend()
    plt.grid(True)
    plt.show()